# Introduction
State notebook purpose here

### Imports
Import libraries and write settings here.

In [2]:
# Notebooks specific imports
from IPython import get_ipython
ipython = get_ipython()
#Expand notebook display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic useful imports
import re
import time
import yaml
from pprint import pprint
from pathlib import Path
import h5py
from datetime import datetime, timedelta

# Data manipulation
import numpy as np
from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.integrate import quad

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import (Circle, RegularPolygon, FancyArrowPatch, ArrowStyle)
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, NullFormatter)
import matplotlib.colors as colors

# Locations
ws_path = Path('/home/alamson/DATA/Chromatin/')
mnt_path = Path.home() / 'projects/DATA/Chromatin/'
ceph_path = Path.home() / 'ceph/DATA/Chromatin/'

/tmp/ipykernel_2756522/1644061892.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Style settings

In [3]:
graph_sty = {
    "axes.titlesize": 20,
    "axes.labelsize": 24,
    "lines.linewidth": 3,
    "lines.markersize": 10,
    "xtick.labelsize": 24,
    "ytick.labelsize": 24,
    "font.size": 20,
    "font.sans-serif": 'Helvetica',
    "text.usetex": False,
    'mathtext.fontset': 'cm',
}
plt.style.use(graph_sty)
        

# Functions

In [41]:
def find_leaf_dirs(path, ignore=['result', 'analysis']):
    is_leaf = True
    leaf_dirs = []
    for p in path.iterdir():
        if p.is_dir() and p.name not in ignore:
            yield from find_leaf_dirs(p, ignore)
            is_leaf = False
    if is_leaf:
        yield path

def get_seed_dirs(path, no_sim_files=False):
    if no_sim_files: # Don't worry if there are sim.* files in leaf dirs
        return list(find_leaf_dirs(path))

    # Fromkeys notation removes duplicates in list
    return list(dict.fromkeys([p.parent for p in path.rglob('*/sim.*')]))

def make_task_file(root_path, sim_dirs):
    db_path = root_path / 'disbatch_files'
    db_path.mkdir(exist_ok=True)
    tfile = db_path / 'Tasks.txt'
    with tfile.open('w') as tf:
        #TODO maybe add header to task file
        for sd in sim_dirs:
            tf.write(f'~/local/bin/syncing_slurm_script.sh {sd} {sd / "batch_script.sh"}\n')
    return tfile

def make_submit_script(tfile, ntasks=1):
    sub_file = tfile.parent / 'submit.sh'
    with sub_file.open('w') as sf:
        sf.write(
f"""sbatch --job-name db_test \\
        --time 1:00:00 \\
        --ntasks {ntasks} \\
        --ntasks-per-node 72 \\
        -o /dev/null \\
        -e /dev/null \\
        --constraint rome \\
        --partition ccb \\
        disBatch -c 1 tasks.txt
""")
    return sub_file

    

# Collection

In [43]:
run_path = ceph_path / '22-08_aLc1_scan6.12_line200-6400_sticky_runs'

sim_dirs = get_seed_dirs(run_path / 'simulations')
tfile = make_task_file(run_path, sim_dirs)
sub_file = make_submit_script(tfile, ntasks=len(sim_dirs))
